# Pipeline Evaluation — Annotation Player

**Listen to classified segments and assign ground-truth labels** for evaluating the full HindiBabyNet pipeline.

Reads `classified_segments.parquet` from Stage 06 and slices audio from the analysis WAV in `audio_processed/`.

### Workflow
1. **Cell 2** — Configure participant, run ID, sampling
2. **Cell 3** — Load parquet + analysis WAV, sample segments
3. **Cell 4** — Annotate: plays each segment, type a shortcut + Enter
4. **Cell 5** — Summary & quick evaluation

### Label shortcuts

| Key | Label | Description |
|-----|-------|-------------|
| `m` | `MAL` | Adult male speech |
| `f` | `FEM` | Adult female speech |
| `c` | `KCHI` | Target child vocalisation |
| `o` | `OCH` | Other child |
| `b` | `SIL` | Background / noise / silence |
| `u` | `OVL` | Unclear / too ambiguous to label |

| Key | Action |
|-----|--------|
| `r` | Replay current segment |
| `p` | Go back (previous segment) |
| `q` | Save & stop (resume later) |

In [ ]:
# ── Configuration ──────────────────────────────────────────────
PARTICIPANT_ID = "ABAN141223"       # ← change this

# Run ID — leave empty to auto-discover latest
RUN_ID = ""  # e.g. "20260217_133307"

# Paths
ARTIFACTS_ROOT    = "../artifacts/runs"
AUDIO_PROCESSED   = "/scratch/users/arunps/hindibabynet/audio_processed"
ANNOTATION_ROOT   = "/scratch/users/arunps/hindibabynet/annotations"

# ── Sampling ──────────────────────────────────────────────────
# Set to 0 to annotate ALL segments (warning: can be thousands)
SAMPLE_PER_CLASS = 50     # random segments per predicted class
SEED             = 42     # for reproducibility
MIN_DURATION     = 0.3    # skip very short segments (seconds)
MAX_DURATION     = 0.0    # 0 = no upper limit
MIN_CONFIDENCE   = 0.0    # 0 = include all confidence levels

In [ ]:
import wave
from pathlib import Path
from typing import List, Optional

import numpy as np
import pandas as pd
import soundfile as sf
import IPython.display as ipd
from IPython.display import display, clear_output

# ── All valid labels ──────────────────────────────────────────
PIPELINE_CLASSES = ["MAL", "FEM", "KCHI", "SIL"]
ALL_LABELS = PIPELINE_CLASSES + ["OCH", "OVL"]

SHORTCUT_MAP = {
    "m": "MAL",
    "f": "FEM",
    "c": "KCHI",
    "o": "OCH",
    "b": "SIL",
    "u": "OVL",
}
SHORTCUT_HELP = "  ".join(f"{k}={v}" for k, v in SHORTCUT_MAP.items())

# ── Discover latest run ──────────────────────────────────────
def find_latest_run(artifacts_root: str) -> str:
    root = Path(artifacts_root)
    runs = sorted(p.name for p in root.iterdir() if p.is_dir())
    assert runs, f"No runs found in {artifacts_root}"
    return runs[-1]

# ── Resolve paths ────────────────────────────────────────────
arts = Path(ARTIFACTS_ROOT)
run_id = RUN_ID or find_latest_run(ARTIFACTS_ROOT)
run_dir = arts / run_id

# Classified segments parquet
pq_path = run_dir / "speaker_classification" / f"{PARTICIPANT_ID}_classified_segments.parquet"
assert pq_path.exists(), f"Parquet not found: {pq_path}"

# Analysis WAV
wav_path = Path(AUDIO_PROCESSED) / PARTICIPANT_ID / f"{PARTICIPANT_ID}.wav"
assert wav_path.exists(), f"Analysis WAV not found: {wav_path}"

# Annotation CSV (persisted between sessions)
ann_dir = Path(ANNOTATION_ROOT) / PARTICIPANT_ID
ann_dir.mkdir(parents=True, exist_ok=True)
ann_csv = ann_dir / f"{PARTICIPANT_ID}_eval_annotations.csv"

print(f"Run ID         : {run_id}")
print(f"Participant    : {PARTICIPANT_ID}")
print(f"Parquet        : {pq_path}")
print(f"Analysis WAV   : {wav_path}")
print(f"Annotation CSV : {ann_csv}")

# ── Load parquet ─────────────────────────────────────────────
df_all = pd.read_parquet(pq_path)
print(f"\nTotal classified segments: {len(df_all)}")
print(f"Class distribution:")
for cls in PIPELINE_CLASSES:
    n = (df_all["predicted_class"] == cls).sum()
    print(f"  {cls:<15}: {n}")

# ── Filter ───────────────────────────────────────────────────
mask = df_all["duration_sec"] >= MIN_DURATION
if MAX_DURATION > 0:
    mask &= df_all["duration_sec"] <= MAX_DURATION
if MIN_CONFIDENCE > 0:
    mask &= df_all["predicted_confidence"] >= MIN_CONFIDENCE
df_filtered = df_all[mask].reset_index(drop=True)
print(f"\nAfter filters (min_dur={MIN_DURATION}s, max_dur={MAX_DURATION or 'none'}, "
      f"min_conf={MIN_CONFIDENCE}): {len(df_filtered)}")

# ── Sample ───────────────────────────────────────────────────
if SAMPLE_PER_CLASS > 0:
    rng = np.random.default_rng(SEED)
    sampled = []
    for cls in PIPELINE_CLASSES:
        cls_df = df_filtered[df_filtered["predicted_class"] == cls]
        n = min(SAMPLE_PER_CLASS, len(cls_df))
        if n > 0:
            idx = rng.choice(len(cls_df), size=n, replace=False)
            sampled.append(cls_df.iloc[idx])
            print(f"  Sampled {cls}: {n}/{len(cls_df)}")
        else:
            print(f"  Sampled {cls}: 0 (none available)")
    df_sample = pd.concat(sampled, ignore_index=True).sort_values("start_sec").reset_index(drop=True)
else:
    df_sample = df_filtered.sort_values("start_sec").reset_index(drop=True)

print(f"\nSegments to annotate: {len(df_sample)}")

# ── Load existing annotations if resuming ────────────────────
if ann_csv.exists():
    df_prev = pd.read_csv(ann_csv)
    # Merge previous human_label into df_sample by matching start_sec + end_sec
    if "human_label" in df_prev.columns:
        prev_map = {
            (round(r["start_sec"], 4), round(r["end_sec"], 4)): r["human_label"]
            for _, r in df_prev.iterrows()
            if pd.notna(r.get("human_label")) and str(r["human_label"]).strip()
        }
        df_sample["human_label"] = df_sample.apply(
            lambda r: prev_map.get((round(r["start_sec"], 4), round(r["end_sec"], 4)), ""),
            axis=1,
        )
        n_restored = (df_sample["human_label"].str.strip() != "").sum()
        if n_restored > 0:
            print(f"Restored {n_restored} previous annotations")
    else:
        df_sample["human_label"] = ""
else:
    df_sample["human_label"] = ""

# Add notes column if missing
if "notes" not in df_sample.columns:
    df_sample["notes"] = ""

# ── Load analysis WAV ────────────────────────────────────────
print(f"\nLoading analysis WAV...")
audio, sr = sf.read(str(wav_path), dtype="float32")
if audio.ndim > 1:
    audio = audio.mean(axis=1)
print(f"Audio: {len(audio)/sr:.1f}s ({len(audio)/sr/3600:.2f}h) @ {sr} Hz")
print(f"\n✓ Ready — run the next cell to start annotating.")

In [ ]:
# ── Annotate: listen + label ──────────────────────────────────
# Re-run this cell to resume from where you left off.

total = len(df_sample)

def _first_unlabelled():
    for i in range(total):
        val = df_sample.at[i, "human_label"]
        if pd.isna(val) or str(val).strip() == "":
            return i
    return total

start_idx = _first_unlabelled()
if start_idx >= total:
    print(f"All {total} segments already annotated!")
    print(df_sample["human_label"].value_counts().to_string())
    print(f"\nRun the next cell for summary.")

def _save():
    df_sample.to_csv(ann_csv, index=False)

def _progress_str():
    done = df_sample["human_label"].notna() & (df_sample["human_label"].str.strip() != "")
    counts = df_sample.loc[done, "human_label"].value_counts()
    parts = [f"{k}={v}" for k, v in sorted(counts.items())]
    return f"{done.sum()}/{total}  |  " + "  ".join(parts)

def _slice_audio(start_sec, end_sec):
    s = max(0, int(round(start_sec * sr)))
    e = min(len(audio), int(round(end_sec * sr)))
    return audio[s:e]

print(f"Progress: {_progress_str()}")
print(f"Starting from segment {start_idx + 1}\n")
print(f"Shortcuts: {SHORTCUT_HELP}")
print(f"Actions:   r=replay  p=previous  q=save & stop\n")

idx = start_idx
try:
    while 0 <= idx < total:
        row = df_sample.iloc[idx]
        pred_cls = row["predicted_class"]
        pred_conf = row["predicted_confidence"]
        dur = row["duration_sec"]
        start_s = row["start_sec"]
        end_s = row["end_sec"]
        existing = row["human_label"]
        has_label = pd.notna(existing) and str(existing).strip() != ""
        tag = f"  [labeled: {existing}]" if has_label else ""

        chunk = _slice_audio(start_s, end_s)

        clear_output(wait=True)
        print(f"[{idx+1}/{total}]  {start_s:.2f}s – {end_s:.2f}s  ({dur:.2f}s)")
        print(f"  Predicted: {pred_cls}  (conf={pred_conf:.3f}){tag}")
        print(f"  Progress: {_progress_str()}")
        print("─" * 55)
        print(f"  {SHORTCUT_HELP}")
        print(f"  r=replay  p=previous  q=save & stop")

        display(ipd.Audio(chunk, rate=sr, autoplay=True))

        while True:
            raw = input("Label: ").strip().lower()
            if raw == "q":
                _save()
                clear_output(wait=True)
                print(f"✓ Saved. {_progress_str()}")
                print(f"  CSV: {ann_csv}")
                print(f"  Re-run this cell to resume.")
                raise StopIteration
            elif raw == "r":
                clear_output(wait=True)
                print(f"[{idx+1}/{total}]  (replaying...)  Predicted: {pred_cls}")
                print("─" * 55)
                display(ipd.Audio(chunk, rate=sr, autoplay=True))
                continue
            elif raw == "p":
                if idx > 0:
                    idx -= 1
                break
            elif raw in SHORTCUT_MAP:
                label = SHORTCUT_MAP[raw]
                df_sample.at[idx, "human_label"] = label
                idx += 1
                done_count = (df_sample["human_label"].notna() & (df_sample["human_label"].str.strip() != "")).sum()
                if done_count % 10 == 0:
                    _save()
                break
            else:
                print(f"  Invalid. Shortcuts: {SHORTCUT_HELP}")

    _save()
    clear_output(wait=True)
    print(f"🎉 All {total} segments annotated!")
    print(f"  {_progress_str()}")
    print(f"  CSV: {ann_csv}")
    print(f"\nRun the next cell for summary & evaluation.")

except StopIteration:
    pass
except KeyboardInterrupt:
    _save()
    print(f"\nInterrupted — saved. Re-run to resume.")

In [ ]:
# ── Summary & quick evaluation ───────────────────────────────

df_fresh = pd.read_csv(ann_csv)
done = df_fresh["human_label"].notna() & (df_fresh["human_label"].str.strip() != "")
annotated = df_fresh[done].copy()

print(f"Annotation CSV : {ann_csv}")
print(f"Annotated      : {len(annotated)}/{len(df_fresh)}\n")

if len(annotated) == 0:
    print("No annotations yet — run the annotation cell first.")
else:
    # Label distribution
    print("── Human label distribution ──")
    for label, count in annotated["human_label"].value_counts().items():
        print(f"  {label:<15}: {count}")

    # Agreement with predictions (4-class pipeline labels only)
    eval_df = annotated[annotated["human_label"].isin(PIPELINE_CLASSES)].copy()
    if len(eval_df) > 0:
        correct = (eval_df["human_label"] == eval_df["predicted_class"]).sum()
        acc = correct / len(eval_df)
        print(f"\n── Pipeline agreement (4-class) ──")
        print(f"  Accuracy: {correct}/{len(eval_df)} = {acc:.1%}")

        # Per-class accuracy
        print(f"\n  Per-class accuracy:")
        for cls in PIPELINE_CLASSES:
            cls_df = eval_df[eval_df["predicted_class"] == cls]
            if len(cls_df) > 0:
                cls_correct = (cls_df["human_label"] == cls).sum()
                print(f"    {cls:<15}: {cls_correct}/{len(cls_df)} = {cls_correct/len(cls_df):.1%}")

        # Confusion matrix
        print(f"\n── Confusion matrix (rows=true, cols=predicted) ──")
        labels_in = sorted(set(eval_df["predicted_class"]) | set(eval_df["human_label"]))
        from sklearn.metrics import confusion_matrix, recall_score
        cm = confusion_matrix(eval_df["human_label"], eval_df["predicted_class"], labels=labels_in)
        cm_df = pd.DataFrame(cm, index=[f"true:{l}" for l in labels_in],
                             columns=[f"pred:{l}" for l in labels_in])
        display(cm_df)

        # UAR
        uar = recall_score(eval_df["human_label"], eval_df["predicted_class"],
                           labels=PIPELINE_CLASSES, average="macro", zero_division=0)
        print(f"\n  UAR (macro recall): {uar:.4f} ({uar*100:.1f}%)")

    # Child sub-classification stats
    n_kchi = (annotated["human_label"] == "KCHI").sum()
    n_och = (annotated["human_label"] == "OCH").sum()
    n_ovl = (annotated["human_label"] == "OVL").sum()
    if n_kchi + n_och > 0:
        print(f"\n── Child sub-classification ──")
        print(f"  KCHI (target)   : {n_kchi}")
        print(f"  OCH  (other)    : {n_och}")
    if n_ovl > 0:
        print(f"\n  OVL (unclear)   : {n_ovl}")

    # Confidence analysis
    if "predicted_confidence" in eval_df.columns and len(eval_df) > 0:
        eval_df = eval_df.copy()
        eval_df["correct"] = eval_df["human_label"] == eval_df["predicted_class"]
        mean_correct = eval_df.loc[eval_df["correct"], "predicted_confidence"].mean()
        mean_wrong = eval_df.loc[~eval_df["correct"], "predicted_confidence"].mean()
        print(f"\n── Confidence analysis ──")
        print(f"  Mean confidence (correct)  : {mean_correct:.4f}")
        if pd.notna(mean_wrong):
            print(f"  Mean confidence (incorrect): {mean_wrong:.4f}")